# [Oregon Statewide ET Project](https://www.dri.edu/project/owrd-et/)


## **Post-Processing Workflow**
 
## Steps: 
1. Concatenate individual static & annual tables (creates one table per year)
2. Join ET Demands data to field summaries
3. Gap-fill EToF using linear interpolation (1 mo) or climatologies (2+ mo)
   * Start/End Year Options:
    > 1985-1991<br>
    > 1992-1997<br>
    > 1998-2003<br>
    > 2004-2009<br>
    > 2010-2015<br>
    > 2016-2022
5. Soil moisture carry forward and applied water calculations
6. HUC8/HUC12 aggregations
<br>


--------

In [17]:
# ----------------------------PARAMETERS---------------------------------------------------------

# path to GEE export folder where tables are stored
table_path = r'E:\dri-owrd-et\tables\ee_exports'

# flag for testing post-processing for a single field
test_flag = True

# define start/end years for exports
start_year = 1985
end_year = 1985

# -----------------------------------------------------------------------------------------------

## import packages

In [22]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

## 1. Concatenate individual static & annual tables

In [23]:

###################################################################

# shapefile location
shp_path = table_path.replace('tables\\ee_exports', 'shapefiles')

# output location
out_path = table_path.replace('ee_exports', 'post_processing\\2_for_et_demands_join')

# list of years to process based on start/end year
year_list = list(range(start_year, end_year+1))

###################################################################

### static attributes
# huc attributes
df_huc = pd.read_csv(os.path.join(table_path, 'or_field_summaries_huc_attributes.csv'), index_col='OPENET_ID')

# annual crop type and gridmet ID attributes 
df_c_pre = pd.read_csv(os.path.join(table_path, 'crop_type_codes_and_gridmet_cells.csv'), index_col='OPENET_ID')

# irrigation system type, irrigation source type, efficiencies, and OWRD admin boundary attributes
df_typ = gpd.read_file(os.path.join(shp_path, 'Oregon_Hyd_Area_Ag_Boundaries_20241016.shp'))[['OPENET_ID', 'ITYPE', 'srctype', 'IRR_EFF', 'OWRD']]
df_typ = df_typ.set_index('OPENET_ID')

# fill blank srctypes and efficiencies with 0's
df_typ.loc[df_typ['srctype'].isnull(), 'srctype'] = 0
df_typ.loc[df_typ['IRR_EFF'].isnull(), 'IRR_EFF'] = 0

# cuenca region attributes
df_cue = pd.read_csv(os.path.join(table_path, 'cuenca_regions.csv'), index_col='OPENET_ID')
df_cue = df_cue.fillna(0)

# bad geometries identified and need to be removed
df_bad = pd.read_csv(os.path.join(table_path, 'bad_geometry_list.csv'), index_col='OPENET_ID')
bad_list = list(df_bad.index)

# only process a single field if test_flag is True
if test_flag:
    print('processing a single field')
    df_huc = df_huc.loc[df_huc.index == 'ORx_155121']
    df_c_pre = df_c_pre.loc[df_c_pre.index == 'ORx_155121']
    df_typ = df_typ.loc[df_typ.index == 'ORx_155121']
    df_cue = df_cue.loc[df_cue.index == 'ORx_155121']
    df_owrd = df_owrd.loc[df_owrd.index == 'ORx_155121']
else:
    print('processing all fields')


# loop through each year
for year in year_list:
    
    # ET dataframe
    df_et = pd.read_csv(os.path.join(table_path, f'or_field_summaries_water_year_shift_1mo_{year}_et.csv'), index_col='OPENET_ID')

    # ET Fraction dataframe
    df_etf = pd.read_csv(os.path.join(table_path, f'or_field_summaries_water_year_shift_1mo_{year}_et_fraction.csv'), index_col='OPENET_ID')
        
    # ET Reference dataframe
    df_eto = pd.read_csv(os.path.join(table_path, f'or_field_summaries_water_year_shift_1mo_{year}_et_reference.csv'),index_col='OPENET_ID')

    # Crop Type and gridmet ID dataframe
    df_c = df_c_pre[[f'CROP_{year}', 'GRIDMET_ID']]

    # precip dataframe
    df_ppt = pd.read_csv(os.path.join(table_path, f'or_field_summaries_water_year_shift_1mo_{year}_ppt.csv'), index_col='OPENET_ID') 
        
    # IrrMapper Irrigated dataframe
    df_irr = pd.read_csv(os.path.join(table_path, f'or_field_summaries_{year}_irrmapper_irrigated.csv'), index_col='OPENET_ID')
    df_irr[f'%_IRRIGATED_{str(year)[2:]}'] = (df_irr['ACRES_IRRIGATED'] / df_irr['ACRES_ALL']) * 100

    # IrrMapper Wetland dataframe
    df_wtl = pd.read_csv(os.path.join(table_path, f'or_field_summaries_{year}_irrmapper_wetland.csv'), index_col='OPENET_ID')
    df_wtl[f'%_WETLAND_{str(year)[2:]}'] = (df_wtl['ACRES_WETLAND'] / df_wtl['ACRES_ALL']) * 100
    df_wtl = df_wtl.drop(columns=['ACRES_ALL'])

    # EToF irrigation status dataframe
    df_etof_irr_status = pd.read_csv(os.path.join(table_path, f'or_field_summaries_{year}_etof_irr_status.csv'), index_col='OPENET_ID')

    # unclassified field nans need to be filled with code 5 for filtering (they are assumed irrigated since they are usually small polygons for single home lawns)
    df_etof_irr_status[f'ETOF_IRR_STATUS_{str(year)[2:]}_MODE'] =  df_etof_irr_status[f'ETOF_IRR_STATUS_{str(year)[2:]}_MODE'].fillna(5)

    # concatenate dataframes on columns using index (unique ID) to match fields
    df = pd.concat([df_huc, df_cue, df_typ, df_c, df_irr, df_wtl, df_etof_irr_status, df_et, df_etf, df_eto, df_ppt], axis=1)

    # filter out bad geometries
    df = df.loc[~df.index.isin(bad_list)]
    
    # reset the index
    df = df.reset_index()

    # export joined dataframe for pairing with ET Demands
    df.to_csv(os.path.join(out_path, f'or_field_summaries_water_year_shift_1mo_{year}_pre_et_demands.csv'), index=False)
        
    print(f'exported dataframe for {year}')

processing a single field
exported dataframe for 1985


## 2. Join ET Demands data to field summaries

In [20]:

# list of years based on start/end parameters
year_list = list(range(start_year, end_year+1))

# CDL - ET Demands crosswalk file
in_path = table_path.replace('ee_exports', 'post_processing\\2_for_et_demands_join')

# ET Demands monthly data path
etd_path = table_path.replace('ee_exports', 'post_processing\\2_for_et_demands_join\\et_demands')

# output path
out_path = table_path.replace('ee_exports', 'post_processing\\3_pre_gap_filled')

# prepare dictionary of crop type codes from CDL and ET Demands
cross_df = pd.read_csv(os.path.join(in_path, 'OR_unique_cdl_etdemands_crosswalk_model_setup.csv'))
cross_dict = dict()
for index, row in cross_df.iterrows():
    cross_dict[row.cdl_no] = list(map(int, str(row.etd_no).split(',')))

    
# function to filter the filenames down to the model we are analyzing
def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]

# loop through each year in the specified list
for year in year_list:

    # monthly openet composite dataframe (each column is monthly value)
    df_et = pd.read_csv(os.path.join(in_path, f'or_field_summaries_water_year_shift_1mo_{year}_pre_et_demands.csv'), index_col='OPENET_ID')

    # create empty columns to fill
    df_et[f'ETD_{str(year)[2:]}'] = np.nan

    df_et[f'ETDa_11_{str(year-1)[2:]}'] = np.nan
    df_et[f'ETDa_12_{str(year-1)[2:]}'] = np.nan
    df_et[f'ETDa_01_{str(year)[2:]}'] = np.nan
    df_et[f'ETDa_02_{str(year)[2:]}'] = np.nan
    df_et[f'ETDa_03_{str(year)[2:]}'] = np.nan
    df_et[f'ETDa_04_{str(year)[2:]}'] = np.nan
    df_et[f'ETDa_05_{str(year)[2:]}'] = np.nan
    df_et[f'ETDa_06_{str(year)[2:]}'] = np.nan
    df_et[f'ETDa_07_{str(year)[2:]}'] = np.nan
    df_et[f'ETDa_08_{str(year)[2:]}'] = np.nan
    df_et[f'ETDa_09_{str(year)[2:]}'] = np.nan
    df_et[f'ETDa_10_{str(year)[2:]}'] = np.nan

    # df_et[f'P_eft_11_{str(year-1)[2:]}'] = np.nan
    # df_et[f'P_eft_12_{str(year-1)[2:]}'] = np.nan 
    # df_et[f'P_eft_01_{str(year)[2:]}'] = np.nan
    # df_et[f'P_eft_02_{str(year)[2:]}'] = np.nan
    # df_et[f'P_eft_03_{str(year)[2:]}'] = np.nan
    # df_et[f'P_eft_04_{str(year)[2:]}'] = np.nan
    # df_et[f'P_eft_05_{str(year)[2:]}'] = np.nan
    # df_et[f'P_eft_06_{str(year)[2:]}'] = np.nan
    # df_et[f'P_eft_07_{str(year)[2:]}'] = np.nan
    # df_et[f'P_eft_08_{str(year)[2:]}'] = np.nan
    # df_et[f'P_eft_09_{str(year)[2:]}'] = np.nan
    # df_et[f'P_eft_10_{str(year)[2:]}'] = np.nan
    
    df_et[f'P_rz_11_{str(year-1)[2:]}'] = np.nan
    df_et[f'P_rz_12_{str(year-1)[2:]}'] = np.nan 
    df_et[f'P_rz_01_{str(year)[2:]}'] = np.nan
    df_et[f'P_rz_02_{str(year)[2:]}'] = np.nan
    df_et[f'P_rz_03_{str(year)[2:]}'] = np.nan
    df_et[f'P_rz_04_{str(year)[2:]}'] = np.nan
    df_et[f'P_rz_05_{str(year)[2:]}'] = np.nan
    df_et[f'P_rz_06_{str(year)[2:]}'] = np.nan
    df_et[f'P_rz_07_{str(year)[2:]}'] = np.nan
    df_et[f'P_rz_08_{str(year)[2:]}'] = np.nan
    df_et[f'P_rz_09_{str(year)[2:]}'] = np.nan
    df_et[f'P_rz_10_{str(year)[2:]}'] = np.nan

    df_et[f'NIWR_11_{str(year-1)[2:]}'] = np.nan
    df_et[f'NIWR_12_{str(year-1)[2:]}'] = np.nan     
    df_et[f'NIWR_01_{str(year)[2:]}'] = np.nan
    df_et[f'NIWR_02_{str(year)[2:]}'] = np.nan
    df_et[f'NIWR_03_{str(year)[2:]}'] = np.nan
    df_et[f'NIWR_04_{str(year)[2:]}'] = np.nan
    df_et[f'NIWR_05_{str(year)[2:]}'] = np.nan
    df_et[f'NIWR_06_{str(year)[2:]}'] = np.nan
    df_et[f'NIWR_07_{str(year)[2:]}'] = np.nan
    df_et[f'NIWR_08_{str(year)[2:]}'] = np.nan
    df_et[f'NIWR_09_{str(year)[2:]}'] = np.nan
    df_et[f'NIWR_10_{str(year)[2:]}'] = np.nan

    for row in df_et.itertuples():

        # crop type for specified year used for crosswalking to ET Demands codes
        cdl_c = df_et[f'CROP_{year}'].at[row.Index]
    
        # gridmet cell ID
        grid_id = int(df_et['GRIDMET_ID'].at[row.Index])
        
        # ET Demands crosswalk value for CDL code
        try:
            etd_c_int = cross_dict[cdl_c][0]
        except Exception as e:
            print(row)
            print(e)
            continue
        etd_c = str(etd_c_int).zfill(2)
        

        # read ET Demands file using gridMET ID and crop type 
        filename = f'{grid_id}_crop_{etd_c}.csv'

        # use these lines instead of the try/except to check if an ET Demands file is missing
        # if os.path.isfile(os.path.join(etd_dir, filename)):
        #     continue
        # else:
        #     print(f'Missing: {filename}')
        #     continue
        
        try:
            df_etd = pd.read_csv(os.path.join(etd_path, filename), header=1, index_col='Date', parse_dates=True)

        except Exception as e:
            print(f'Missing: {filename}')
            # print(e)
            continue

        # filter df to specified year
        df_etd_f = df_etd[df_etd['Year'].isin([(year-1), year])]

        df_et[f'ETD_{str(year)[2:]}'].at[row.Index] = etd_c
        
        df_et[f'ETDa_11_{str(year-1)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 11) & (df_etd_f['Year'] == (year-1))]['ETact'].iloc[0]
        df_et[f'ETDa_12_{str(year-1)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 12) & (df_etd_f['Year'] == (year-1))]['ETact'].iloc[0]
        df_et[f'ETDa_01_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 1) & (df_etd_f['Year'] == year)]['ETact'].iloc[0]
        df_et[f'ETDa_02_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 2) & (df_etd_f['Year'] == year)]['ETact'].iloc[0]
        df_et[f'ETDa_03_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 3) & (df_etd_f['Year'] == year)]['ETact'].iloc[0]
        df_et[f'ETDa_04_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 4) & (df_etd_f['Year'] == year)]['ETact'].iloc[0]
        df_et[f'ETDa_05_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 5) & (df_etd_f['Year'] == year)]['ETact'].iloc[0]
        df_et[f'ETDa_06_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 6) & (df_etd_f['Year'] == year)]['ETact'].iloc[0]
        df_et[f'ETDa_07_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 7) & (df_etd_f['Year'] == year)]['ETact'].iloc[0]
        df_et[f'ETDa_08_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 8) & (df_etd_f['Year'] == year)]['ETact'].iloc[0]
        df_et[f'ETDa_09_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 9) & (df_etd_f['Year'] == year)]['ETact'].iloc[0]
        df_et[f'ETDa_10_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 10) & (df_etd_f['Year'] == year)]['ETact'].iloc[0]

        # df_et[f'P_eft_11_{str(year-1)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 11) & (df_etd_f['Year'] == (year-1))]['P_eft'].iloc[0]
        # df_et[f'P_eft_12_{str(year-1)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 12) & (df_etd_f['Year'] == (year-1))]['P_eft'].iloc[0]
        # df_et[f'P_eft_01_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 1) & (df_etd_f['Year'] == year)]['P_eft'].iloc[0]
        # df_et[f'P_eft_02_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 2) & (df_etd_f['Year'] == year)]['P_eft'].iloc[0]
        # df_et[f'P_eft_03_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 3) & (df_etd_f['Year'] == year)]['P_eft'].iloc[0]
        # df_et[f'P_eft_04_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 4) & (df_etd_f['Year'] == year)]['P_eft'].iloc[0]
        # df_et[f'P_eft_05_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 5) & (df_etd_f['Year'] == year)]['P_eft'].iloc[0]
        # df_et[f'P_eft_06_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 6) & (df_etd_f['Year'] == year)]['P_eft'].iloc[0]
        # df_et[f'P_eft_07_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 7) & (df_etd_f['Year'] == year)]['P_eft'].iloc[0]
        # df_et[f'P_eft_08_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 8) & (df_etd_f['Year'] == year)]['P_eft'].iloc[0]
        # df_et[f'P_eft_09_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 9) & (df_etd_f['Year'] == year)]['P_eft'].iloc[0]
        # df_et[f'P_eft_10_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 10) & (df_etd_f['Year'] == year)]['P_eft'].iloc[0]

        df_et[f'P_rz_11_{str(year-1)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 11) & (df_etd_f['Year'] == (year-1))]['P_rz'].iloc[0]
        df_et[f'P_rz_12_{str(year-1)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 12) & (df_etd_f['Year'] == (year-1))]['P_rz'].iloc[0]
        df_et[f'P_rz_01_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 1) & (df_etd_f['Year'] == year)]['P_rz'].iloc[0]
        df_et[f'P_rz_02_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 2) & (df_etd_f['Year'] == year)]['P_rz'].iloc[0]
        df_et[f'P_rz_03_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 3) & (df_etd_f['Year'] == year)]['P_rz'].iloc[0]
        df_et[f'P_rz_04_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 4) & (df_etd_f['Year'] == year)]['P_rz'].iloc[0]
        df_et[f'P_rz_05_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 5) & (df_etd_f['Year'] == year)]['P_rz'].iloc[0]
        df_et[f'P_rz_06_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 6) & (df_etd_f['Year'] == year)]['P_rz'].iloc[0]
        df_et[f'P_rz_07_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 7) & (df_etd_f['Year'] == year)]['P_rz'].iloc[0]
        df_et[f'P_rz_08_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 8) & (df_etd_f['Year'] == year)]['P_rz'].iloc[0]
        df_et[f'P_rz_09_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 9) & (df_etd_f['Year'] == year)]['P_rz'].iloc[0]
        df_et[f'P_rz_10_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 10) & (df_etd_f['Year'] == year)]['P_rz'].iloc[0]

        df_et[f'NIWR_11_{str(year-1)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 11) & (df_etd_f['Year'] == (year-1))]['NIWR'].iloc[0]
        df_et[f'NIWR_12_{str(year-1)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 12) & (df_etd_f['Year'] == (year-1))]['NIWR'].iloc[0]
        df_et[f'NIWR_01_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 1) & (df_etd_f['Year'] == year)]['NIWR'].iloc[0]
        df_et[f'NIWR_02_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 2) & (df_etd_f['Year'] == year)]['NIWR'].iloc[0]
        df_et[f'NIWR_03_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 3) & (df_etd_f['Year'] == year)]['NIWR'].iloc[0]
        df_et[f'NIWR_04_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 4) & (df_etd_f['Year'] == year)]['NIWR'].iloc[0]
        df_et[f'NIWR_05_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 5) & (df_etd_f['Year'] == year)]['NIWR'].iloc[0]
        df_et[f'NIWR_06_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 6) & (df_etd_f['Year'] == year)]['NIWR'].iloc[0]
        df_et[f'NIWR_07_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 7) & (df_etd_f['Year'] == year)]['NIWR'].iloc[0]
        df_et[f'NIWR_08_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 8) & (df_etd_f['Year'] == year)]['NIWR'].iloc[0]
        df_et[f'NIWR_09_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 9) & (df_etd_f['Year'] == year)]['NIWR'].iloc[0]
        df_et[f'NIWR_10_{str(year)[2:]}'].at[row.Index] = df_etd_f.loc[(df_etd_f['Month'] == 10) & (df_etd_f['Year'] == year)]['NIWR'].iloc[0]

    df_et.to_csv(os.path.join(out_path, f'or_openet_etdemands_monthly_water_year_shift_1mo_{year}_pre_gapfill.csv'))
    
    print(f'finished processing all files for {year}')

finished processing all files for 1985


## 3. Gap-fill EToF using linear interpolation (1 mo) or climatologies (2+ mo)

* Start/End Year Options:
> 1985-1991<br>
> 1992-1997<br>
> 1998-2003<br>
> 2004-2009<br>
> 2010-2015<br>
> 2016-2022

In [ ]:

yr_list = np.arange(start_year, end_year+1)
yr_abr_list = [int(str(yr)[2:]) for yr in yr_list]

eff_ppt_var = 'P_rz'

# input path
in_path = table_path.replace('ee_exports', 'post_processing\\3_pre_gap_filled')

# output path
out_path = table_path.replace('ee_exports', 'post_processing\\4_gap_filled')

# EToF Climatology  dataframe to gap-fill if multiple adjacent-months missing (also first and last values)
if (yr_list[0] == 1985 and yr_list[-1] == 1991):
    df_c = pd.read_csv(os.path.join(table_path, f'or_field_summaries_water_year_shift_1mo_1984_{yr_list[-1]}_et_fraction_climo.csv'), index_col='OPENET_ID')
elif (yr_list[0] == 2016 and yr_list[-1] == 2022):
    df_c = pd.read_csv(os.path.join(table_path, f'or_field_summaries_water_year_shift_1mo_{yr_list[0]}_2021_et_fraction_climo.csv'), index_col='OPENET_ID')
else:
    df_c = pd.read_csv(os.path.join(table_path, f'or_field_summaries_water_year_shift_1mo_{yr_list[0]}_{yr_list[-1]}_et_fraction_climo.csv'), index_col='OPENET_ID')

# rename columns for climo file
df_c.columns = ['ETc_Fraction_11','ETc_Fraction_12','ETc_Fraction_01','ETc_Fraction_02','ETc_Fraction_03','ETc_Fraction_04','ETc_Fraction_05','ETc_Fraction_06',
                'ETc_Fraction_07','ETc_Fraction_08','ETc_Fraction_09','ETc_Fraction_10']

# read all years into dataframes in order to concatenate and gap fill properly
if (yr_list[0] == 1985 and yr_list[-1] == 1991):
    df1 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_1985_pre_gapfill.csv'), index_col='OPENET_ID')
    df1 = df1.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[0]:02d}'})
    df2 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_1986_pre_gapfill.csv'), index_col='OPENET_ID')
    df2 = df2.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[1]:02d}'})
    df3 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_1987_pre_gapfill.csv'), index_col='OPENET_ID')
    df3 = df3.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[2]:02d}'})
    df4 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_1988_pre_gapfill.csv'), index_col='OPENET_ID')
    df4 = df4.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[3]:02d}'})
    df5 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_1989_pre_gapfill.csv'), index_col='OPENET_ID')
    df5 = df5.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[4]:02d}'})
    df6 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_1990_pre_gapfill.csv'), index_col='OPENET_ID')
    df6 = df6.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[5]:02d}'})
    df7 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_1991_pre_gapfill.csv'), index_col='OPENET_ID')
    df7 = df7.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[6]:02d}'})

    df = pd.concat([df_c, df1, df2, df3, df4, df5, df6, df7], axis=1)
elif (yr_list[0] == 2016 and yr_list[-1] == 2022):
    df1 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_2016_pre_gapfill.csv'), index_col='OPENET_ID')
    df1 = df1.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[0]:02d}'})
    df2 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_2017_pre_gapfill.csv'), index_col='OPENET_ID')
    df2 = df2.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[1]:02d}'})
    df3 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_2018_pre_gapfill.csv'), index_col='OPENET_ID')
    df3 = df3.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[2]:02d}'})
    df4 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_2019_pre_gapfill.csv'), index_col='OPENET_ID')
    df4 = df4.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[3]:02d}'})
    df5 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_2020_pre_gapfill.csv'), index_col='OPENET_ID')
    df5 = df5.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[4]:02d}'})
    df6 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_2021_pre_gapfill.csv'), index_col='OPENET_ID')
    df6 = df6.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[5]:02d}'})
    df7 = pd.read_csv(os.path.join(in_path, 'or_openet_etdemands_monthly_water_year_shift_1mo_2022_pre_gapfill.csv'), index_col='OPENET_ID')
    df7 = df7.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[6]:02d}'})

    df = pd.concat([df_c, df1, df2, df3, df4, df5, df6, df7], axis=1)
else:
    df1 = pd.read_csv(os.path.join(main_dir, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[0]}_pre_gapfill.csv'), index_col='OPENET_ID')
    df1 = df1.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[0]:02d}'})
    df2 = pd.read_csv(os.path.join(main_dir, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[1]}_pre_gapfill.csv'), index_col='OPENET_ID')
    df2 = df2.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[1]:02d}'})
    df3 = pd.read_csv(os.path.join(main_dir, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[2]}_pre_gapfill.csv'), index_col='OPENET_ID')
    df3 = df3.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[2]:02d}'})
    df4 = pd.read_csv(os.path.join(main_dir, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[3]}_pre_gapfill.csv'), index_col='OPENET_ID')
    df4 = df4.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[3]:02d}'})
    df5 = pd.read_csv(os.path.join(main_dir, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[4]}_pre_gapfill.csv'), index_col='OPENET_ID')
    df5 = df5.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[4]:02d}'})
    df6 = pd.read_csv(os.path.join(main_dir, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[5]}_pre_gapfill.csv'), index_col='OPENET_ID')
    df6 = df6.rename(columns={'ACRES_FTR_GEOM': f'ACRES_FTR_GEOM_{yr_abr_list[5]:02d}'})

    df = pd.concat([df_c, df1, df2, df3, df4, df5, df6], axis=1)

# Last month of period needs to be filled with climo before linear interpolation (first month does not since interp doesn't catch it)
df[f'ET_Fraction_10_{yr_abr_list[-1]:02d}'] = df[f'ET_Fraction_10_{yr_abr_list[-1]:02d}'].fillna(df['ETc_Fraction_10'])

# linearly interpolate isolated monthly nans/gaps 
# df_t = df.loc[:,df.columns.str.contains('ET_Fraction')]
# df.loc[:,df.columns.str.contains('ET_Fraction')] = df.loc[:,df.columns.str.contains('ET_Fraction')].interpolate(method='linear',limit=1,limit_area='inside',axis=1)

# linearly interpolate isolated monthly nans/gaps only, not consecutive nans
fval = (df.loc[:, df.columns.str.contains('ET_Fraction')].shift(1, axis=1).add(df.loc[:, df.columns.str.contains('ET_Fraction')].shift(-1, axis=1)) / 2)
df.loc[:, df.columns.str.contains('ET_Fraction')] = df.loc[:, df.columns.str.contains('ET_Fraction')].fillna(value=fval, axis=1)

# fill non-isolated (i.e., consecutive/adjacent) monthly nans/gaps with the climo values explicitly
for yr in yr_abr_list:
    print(f'gap filling {yr}')

    # fill the rest of the nans (consecutive nans) with the climatologies
    # the year 2000 has to have this special condition for subtracting 1 from 0 (2000 actual year value)
    if yr == 0:
        df['ET_Fraction_11_99'] = df['ET_Fraction_11_99'].fillna(df['ETc_Fraction_11'])
        df['ET_Fraction_12_99'] = df['ET_Fraction_12_99'].fillna(df['ETc_Fraction_12'])
    else:
        df[f'ET_Fraction_11_{yr-1:02d}'] = df[f'ET_Fraction_11_{yr-1:02d}'].fillna(df['ETc_Fraction_11'])
        df[f'ET_Fraction_12_{yr-1:02d}'] = df[f'ET_Fraction_12_{yr-1:02d}'].fillna(df['ETc_Fraction_12'])
    df[f'ET_Fraction_01_{yr:02d}'] = df[f'ET_Fraction_01_{yr:02d}'].fillna(df['ETc_Fraction_01'])
    df[f'ET_Fraction_02_{yr:02d}'] = df[f'ET_Fraction_02_{yr:02d}'].fillna(df['ETc_Fraction_02'])
    df[f'ET_Fraction_03_{yr:02d}'] = df[f'ET_Fraction_03_{yr:02d}'].fillna(df['ETc_Fraction_03'])
    df[f'ET_Fraction_04_{yr:02d}'] = df[f'ET_Fraction_04_{yr:02d}'].fillna(df['ETc_Fraction_04'])
    df[f'ET_Fraction_05_{yr:02d}'] = df[f'ET_Fraction_05_{yr:02d}'].fillna(df['ETc_Fraction_05'])
    df[f'ET_Fraction_06_{yr:02d}'] = df[f'ET_Fraction_06_{yr:02d}'].fillna(df['ETc_Fraction_06'])
    df[f'ET_Fraction_07_{yr:02d}'] = df[f'ET_Fraction_07_{yr:02d}'].fillna(df['ETc_Fraction_07'])
    df[f'ET_Fraction_08_{yr:02d}'] = df[f'ET_Fraction_08_{yr:02d}'].fillna(df['ETc_Fraction_08'])
    df[f'ET_Fraction_09_{yr:02d}'] = df[f'ET_Fraction_09_{yr:02d}'].fillna(df['ETc_Fraction_09'])
    df[f'ET_Fraction_10_{yr:02d}'] = df[f'ET_Fraction_10_{yr:02d}'].fillna(df['ETc_Fraction_10'])

    # some fields' EToF climos for Dec 1984 were missing so need to interpolate those months after above gap-filling
    if yr == 85:
        df.loc[:,df.columns.str.contains('ET_Fraction')] = df.loc[:,df.columns.str.contains('ET_Fraction')].interpolate(method='linear', axis=1)        

    # fill nans in actual et with the gap-filled et fraction * et reference
    if yr == 0:
        df[f'ETa_11_99'] = df[f'ETa_11_99'].fillna(df[f'ET_Fraction_11_99'] * df[f'ET_Reference_11_99'])
        df[f'ETa_12_99'] = df[f'ETa_12_99'].fillna(df[f'ET_Fraction_12_99'] * df[f'ET_Reference_12_99'])
    else:  
        df[f'ETa_11_{yr-1:02d}'] = df[f'ETa_11_{yr-1:02d}'].fillna(df[f'ET_Fraction_11_{yr-1:02d}'] * df[f'ET_Reference_11_{yr-1:02d}'])
        df[f'ETa_12_{yr-1:02d}'] = df[f'ETa_12_{yr-1:02d}'].fillna(df[f'ET_Fraction_12_{yr-1:02d}'] * df[f'ET_Reference_12_{yr-1:02d}'])
    df[f'ETa_01_{yr:02d}'] = df[f'ETa_01_{yr:02d}'].fillna(df[f'ET_Fraction_01_{yr:02d}'] * df[f'ET_Reference_01_{yr:02d}'])
    df[f'ETa_02_{yr:02d}'] = df[f'ETa_02_{yr:02d}'].fillna(df[f'ET_Fraction_02_{yr:02d}'] * df[f'ET_Reference_02_{yr:02d}'])
    df[f'ETa_03_{yr:02d}'] = df[f'ETa_03_{yr:02d}'].fillna(df[f'ET_Fraction_03_{yr:02d}'] * df[f'ET_Reference_03_{yr:02d}'])
    df[f'ETa_04_{yr:02d}'] = df[f'ETa_04_{yr:02d}'].fillna(df[f'ET_Fraction_04_{yr:02d}'] * df[f'ET_Reference_04_{yr:02d}'])
    df[f'ETa_05_{yr:02d}'] = df[f'ETa_05_{yr:02d}'].fillna(df[f'ET_Fraction_05_{yr:02d}'] * df[f'ET_Reference_05_{yr:02d}'])
    df[f'ETa_06_{yr:02d}'] = df[f'ETa_06_{yr:02d}'].fillna(df[f'ET_Fraction_06_{yr:02d}'] * df[f'ET_Reference_06_{yr:02d}'])
    df[f'ETa_07_{yr:02d}'] = df[f'ETa_07_{yr:02d}'].fillna(df[f'ET_Fraction_07_{yr:02d}'] * df[f'ET_Reference_07_{yr:02d}'])
    df[f'ETa_08_{yr:02d}'] = df[f'ETa_08_{yr:02d}'].fillna(df[f'ET_Fraction_08_{yr:02d}'] * df[f'ET_Reference_08_{yr:02d}'])
    df[f'ETa_09_{yr:02d}'] = df[f'ETa_09_{yr:02d}'].fillna(df[f'ET_Fraction_09_{yr:02d}'] * df[f'ET_Reference_09_{yr:02d}'])
    df[f'ETa_10_{yr:02d}'] = df[f'ETa_10_{yr:02d}'].fillna(df[f'ET_Fraction_10_{yr:02d}'] * df[f'ET_Reference_10_{yr:02d}'])
    
    # convert units from mm to inches 
    if yr == 0:
        df[f'ETa_11_99_in'] = df[f'ETa_11_99'] / 25.4
        df[f'ETa_12_99_in'] = df[f'ETa_12_99'] / 25.4
    else:
        df[f'ETa_11_{yr-1:02d}_in'] = df[f'ETa_11_{yr-1:02d}'] / 25.4
        df[f'ETa_12_{yr-1:02d}_in'] = df[f'ETa_12_{yr-1:02d}'] / 25.4
    df[f'ETa_01_{yr:02d}_in'] = df[f'ETa_01_{yr:02d}'] / 25.4
    df[f'ETa_02_{yr:02d}_in'] = df[f'ETa_02_{yr:02d}'] / 25.4
    df[f'ETa_03_{yr:02d}_in'] = df[f'ETa_03_{yr:02d}'] / 25.4
    df[f'ETa_04_{yr:02d}_in'] = df[f'ETa_04_{yr:02d}'] / 25.4
    df[f'ETa_05_{yr:02d}_in'] = df[f'ETa_05_{yr:02d}'] / 25.4
    df[f'ETa_06_{yr:02d}_in'] = df[f'ETa_06_{yr:02d}'] / 25.4
    df[f'ETa_07_{yr:02d}_in'] = df[f'ETa_07_{yr:02d}'] / 25.4
    df[f'ETa_08_{yr:02d}_in'] = df[f'ETa_08_{yr:02d}'] / 25.4
    df[f'ETa_09_{yr:02d}_in'] = df[f'ETa_09_{yr:02d}'] / 25.4
    df[f'ETa_10_{yr:02d}_in'] = df[f'ETa_10_{yr:02d}'] / 25.4
    
    # convert units from mm to inches 
    if yr == 0:
        df[f'ETDa_11_99_in'] = df[f'ETDa_11_99'] / 25.4
        df[f'ETDa_12_99_in'] = df[f'ETDa_12_99'] / 25.4
    else:
        df[f'ETDa_11_{yr-1:02d}_in'] = df[f'ETDa_11_{yr-1:02d}'] / 25.4
        df[f'ETDa_12_{yr-1:02d}_in'] = df[f'ETDa_12_{yr-1:02d}'] / 25.4
    df[f'ETDa_01_{yr:02d}_in'] = df[f'ETDa_01_{yr:02d}'] / 25.4
    df[f'ETDa_02_{yr:02d}_in'] = df[f'ETDa_02_{yr:02d}'] / 25.4
    df[f'ETDa_03_{yr:02d}_in'] = df[f'ETDa_03_{yr:02d}'] / 25.4
    df[f'ETDa_04_{yr:02d}_in'] = df[f'ETDa_04_{yr:02d}'] / 25.4
    df[f'ETDa_05_{yr:02d}_in'] = df[f'ETDa_05_{yr:02d}'] / 25.4
    df[f'ETDa_06_{yr:02d}_in'] = df[f'ETDa_06_{yr:02d}'] / 25.4
    df[f'ETDa_07_{yr:02d}_in'] = df[f'ETDa_07_{yr:02d}'] / 25.4
    df[f'ETDa_08_{yr:02d}_in'] = df[f'ETDa_08_{yr:02d}'] / 25.4
    df[f'ETDa_09_{yr:02d}_in'] = df[f'ETDa_09_{yr:02d}'] / 25.4
    df[f'ETDa_10_{yr:02d}_in'] = df[f'ETDa_10_{yr:02d}'] / 25.4

    if yr == 0:
        df[f'ET_Reference_11_99_in'] = df[f'ET_Reference_11_99'] / 25.4
        df[f'ET_Reference_12_99_in'] = df[f'ET_Reference_12_99'] / 25.4
    else:
        df[f'ET_Reference_11_{yr-1:02d}_in'] = df[f'ET_Reference_11_{yr-1:02d}'] / 25.4
        df[f'ET_Reference_12_{yr-1:02d}_in'] = df[f'ET_Reference_12_{yr-1:02d}'] / 25.4
    df[f'ET_Reference_01_{yr:02d}_in'] = df[f'ET_Reference_01_{yr:02d}'] / 25.4
    df[f'ET_Reference_02_{yr:02d}_in'] = df[f'ET_Reference_02_{yr:02d}'] / 25.4
    df[f'ET_Reference_03_{yr:02d}_in'] = df[f'ET_Reference_03_{yr:02d}'] / 25.4
    df[f'ET_Reference_04_{yr:02d}_in'] = df[f'ET_Reference_04_{yr:02d}'] / 25.4
    df[f'ET_Reference_05_{yr:02d}_in'] = df[f'ET_Reference_05_{yr:02d}'] / 25.4
    df[f'ET_Reference_06_{yr:02d}_in'] = df[f'ET_Reference_06_{yr:02d}'] / 25.4
    df[f'ET_Reference_07_{yr:02d}_in'] = df[f'ET_Reference_07_{yr:02d}'] / 25.4
    df[f'ET_Reference_08_{yr:02d}_in'] = df[f'ET_Reference_08_{yr:02d}'] / 25.4
    df[f'ET_Reference_09_{yr:02d}_in'] = df[f'ET_Reference_09_{yr:02d}'] / 25.4
    df[f'ET_Reference_10_{yr:02d}_in'] = df[f'ET_Reference_10_{yr:02d}'] / 25.4
    
    if yr == 0:
        df[f'PPT_11_99_in'] = df[f'PPT_11_99'] / 25.4
        df[f'PPT_12_99_in'] = df[f'PPT_12_99'] / 25.4
    else:
        df[f'PPT_11_{yr-1:02d}_in'] = df[f'PPT_11_{yr-1:02d}'] / 25.4
        df[f'PPT_12_{yr-1:02d}_in'] = df[f'PPT_12_{yr-1:02d}'] / 25.4
    df[f'PPT_01_{yr:02d}_in'] = df[f'PPT_01_{yr:02d}'] / 25.4
    df[f'PPT_02_{yr:02d}_in'] = df[f'PPT_02_{yr:02d}'] / 25.4
    df[f'PPT_03_{yr:02d}_in'] = df[f'PPT_03_{yr:02d}'] / 25.4
    df[f'PPT_04_{yr:02d}_in'] = df[f'PPT_04_{yr:02d}'] / 25.4
    df[f'PPT_05_{yr:02d}_in'] = df[f'PPT_05_{yr:02d}'] / 25.4
    df[f'PPT_06_{yr:02d}_in'] = df[f'PPT_06_{yr:02d}'] / 25.4
    df[f'PPT_07_{yr:02d}_in'] = df[f'PPT_07_{yr:02d}'] / 25.4
    df[f'PPT_08_{yr:02d}_in'] = df[f'PPT_08_{yr:02d}'] / 25.4
    df[f'PPT_09_{yr:02d}_in'] = df[f'PPT_09_{yr:02d}'] / 25.4
    df[f'PPT_10_{yr:02d}_in'] = df[f'PPT_10_{yr:02d}'] / 25.4
    
    # if yr == 0:
    #     df[f'P_eft_11_99_in'] = df[f'P_eft_11_99'] / 25.4
    #     df[f'P_eft_12_99_in'] = df[f'P_eft_12_99'] / 25.4    
    # else:
    #     df[f'P_eft_11_{yr-1:02d}_in'] = df[f'P_eft_11_{yr-1:02d}'] / 25.4
    #     df[f'P_eft_12_{yr-1:02d}_in'] = df[f'P_eft_12_{yr-1:02d}'] / 25.4
    # df[f'P_eft_01_{yr:02d}_in'] = df[f'P_eft_01_{yr:02d}'] / 25.4
    # df[f'P_eft_02_{yr:02d}_in'] = df[f'P_eft_02_{yr:02d}'] / 25.4
    # df[f'P_eft_03_{yr:02d}_in'] = df[f'P_eft_03_{yr:02d}'] / 25.4
    # df[f'P_eft_04_{yr:02d}_in'] = df[f'P_eft_04_{yr:02d}'] / 25.4
    # df[f'P_eft_05_{yr:02d}_in'] = df[f'P_eft_05_{yr:02d}'] / 25.4
    # df[f'P_eft_06_{yr:02d}_in'] = df[f'P_eft_06_{yr:02d}'] / 25.4
    # df[f'P_eft_07_{yr:02d}_in'] = df[f'P_eft_07_{yr:02d}'] / 25.4
    # df[f'P_eft_08_{yr:02d}_in'] = df[f'P_eft_08_{yr:02d}'] / 25.4
    # df[f'P_eft_09_{yr:02d}_in'] = df[f'P_eft_09_{yr:02d}'] / 25.4
    # df[f'P_eft_10_{yr:02d}_in'] = df[f'P_eft_10_{yr:02d}'] / 25.4

    if yr == 0:
        df[f'P_rz_11_99_in'] = df[f'P_rz_11_99'] / 25.4
        df[f'P_rz_12_99_in'] = df[f'P_rz_12_99'] / 25.4  
    else:
        df[f'P_rz_11_{yr-1:02d}_in'] = df[f'P_rz_11_{yr-1:02d}'] / 25.4
        df[f'P_rz_12_{yr-1:02d}_in'] = df[f'P_rz_12_{yr-1:02d}'] / 25.4
    df[f'P_rz_01_{yr:02d}_in'] = df[f'P_rz_01_{yr:02d}'] / 25.4
    df[f'P_rz_02_{yr:02d}_in'] = df[f'P_rz_02_{yr:02d}'] / 25.4
    df[f'P_rz_03_{yr:02d}_in'] = df[f'P_rz_03_{yr:02d}'] / 25.4
    df[f'P_rz_04_{yr:02d}_in'] = df[f'P_rz_04_{yr:02d}'] / 25.4
    df[f'P_rz_05_{yr:02d}_in'] = df[f'P_rz_05_{yr:02d}'] / 25.4
    df[f'P_rz_06_{yr:02d}_in'] = df[f'P_rz_06_{yr:02d}'] / 25.4
    df[f'P_rz_07_{yr:02d}_in'] = df[f'P_rz_07_{yr:02d}'] / 25.4
    df[f'P_rz_08_{yr:02d}_in'] = df[f'P_rz_08_{yr:02d}'] / 25.4
    df[f'P_rz_09_{yr:02d}_in'] = df[f'P_rz_09_{yr:02d}'] / 25.4
    df[f'P_rz_10_{yr:02d}_in'] = df[f'P_rz_10_{yr:02d}'] / 25.4

    if yr == 0:
        df[f'NIWR_11_99_in'] = (df[f'NIWR_11_99'] / 25.4)
        df[f'NIWR_12_99_in'] = (df[f'NIWR_12_99'] / 25.4)
    else:
        df[f'NIWR_11_{yr-1:02d}_in'] = (df[f'NIWR_11_{yr-1:02d}'] / 25.4)
        df[f'NIWR_12_{yr-1:02d}_in'] = (df[f'NIWR_12_{yr-1:02d}'] / 25.4)
    df[f'NIWR_01_{yr:02d}_in'] = (df[f'NIWR_01_{yr:02d}'] / 25.4)
    df[f'NIWR_02_{yr:02d}_in'] = (df[f'NIWR_02_{yr:02d}'] / 25.4)
    df[f'NIWR_03_{yr:02d}_in'] = (df[f'NIWR_03_{yr:02d}'] / 25.4)
    df[f'NIWR_04_{yr:02d}_in'] = (df[f'NIWR_04_{yr:02d}'] / 25.4)
    df[f'NIWR_05_{yr:02d}_in'] = (df[f'NIWR_05_{yr:02d}'] / 25.4)
    df[f'NIWR_06_{yr:02d}_in'] = (df[f'NIWR_06_{yr:02d}'] / 25.4)
    df[f'NIWR_07_{yr:02d}_in'] = (df[f'NIWR_07_{yr:02d}'] / 25.4)
    df[f'NIWR_08_{yr:02d}_in'] = (df[f'NIWR_08_{yr:02d}'] / 25.4)
    df[f'NIWR_09_{yr:02d}_in'] = (df[f'NIWR_09_{yr:02d}'] / 25.4)
    df[f'NIWR_10_{yr:02d}_in'] = (df[f'NIWR_10_{yr:02d}'] / 25.4)
    
    # calculate volumes of each monthly value for all variables
    if yr == 0:
        df[f'ET_VOLUME_11_99_acft'] = (df[f'ETa_11_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
        df[f'ET_VOLUME_12_99_acft'] = (df[f'ETa_12_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']        
    else:
        df[f'ET_VOLUME_11_{yr-1:02d}_acft'] = (df[f'ETa_11_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
        df[f'ET_VOLUME_12_{yr-1:02d}_acft'] = (df[f'ETa_12_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ET_VOLUME_01_{yr:02d}_acft'] = (df[f'ETa_01_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ET_VOLUME_02_{yr:02d}_acft'] = (df[f'ETa_02_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ET_VOLUME_03_{yr:02d}_acft'] = (df[f'ETa_03_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ET_VOLUME_04_{yr:02d}_acft'] = (df[f'ETa_04_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ET_VOLUME_05_{yr:02d}_acft'] = (df[f'ETa_05_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ET_VOLUME_06_{yr:02d}_acft'] = (df[f'ETa_06_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ET_VOLUME_07_{yr:02d}_acft'] = (df[f'ETa_07_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ET_VOLUME_08_{yr:02d}_acft'] = (df[f'ETa_08_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ET_VOLUME_09_{yr:02d}_acft'] = (df[f'ETa_09_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ET_VOLUME_10_{yr:02d}_acft'] = (df[f'ETa_10_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    
    # calculate volumes of each monthly value for all variables
    if yr == 0:
        df[f'ETDa_VOLUME_11_99_acft'] = (df[f'ETDa_11_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
        df[f'ETDa_VOLUME_12_99_acft'] = (df[f'ETDa_12_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    else:
        df[f'ETDa_VOLUME_11_{yr-1:02d}_acft'] = (df[f'ETDa_11_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
        df[f'ETDa_VOLUME_12_{yr-1:02d}_acft'] = (df[f'ETDa_12_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETDa_VOLUME_01_{yr:02d}_acft'] = (df[f'ETDa_01_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETDa_VOLUME_02_{yr:02d}_acft'] = (df[f'ETDa_02_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETDa_VOLUME_03_{yr:02d}_acft'] = (df[f'ETDa_03_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETDa_VOLUME_04_{yr:02d}_acft'] = (df[f'ETDa_04_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETDa_VOLUME_05_{yr:02d}_acft'] = (df[f'ETDa_05_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETDa_VOLUME_06_{yr:02d}_acft'] = (df[f'ETDa_06_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETDa_VOLUME_07_{yr:02d}_acft'] = (df[f'ETDa_07_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETDa_VOLUME_08_{yr:02d}_acft'] = (df[f'ETDa_08_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETDa_VOLUME_09_{yr:02d}_acft'] = (df[f'ETDa_09_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETDa_VOLUME_10_{yr:02d}_acft'] = (df[f'ETDa_10_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']

    if yr == 0:
        df[f'ETO_VOLUME_11_99_acft'] = (df[f'ET_Reference_11_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
        df[f'ETO_VOLUME_12_99_acft'] = (df[f'ET_Reference_12_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    else:
        df[f'ETO_VOLUME_11_{yr-1:02d}_acft'] = (df[f'ET_Reference_11_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
        df[f'ETO_VOLUME_12_{yr-1:02d}_acft'] = (df[f'ET_Reference_12_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETO_VOLUME_01_{yr:02d}_acft'] = (df[f'ET_Reference_01_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETO_VOLUME_02_{yr:02d}_acft'] = (df[f'ET_Reference_02_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETO_VOLUME_03_{yr:02d}_acft'] = (df[f'ET_Reference_03_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETO_VOLUME_04_{yr:02d}_acft'] = (df[f'ET_Reference_04_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETO_VOLUME_05_{yr:02d}_acft'] = (df[f'ET_Reference_05_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETO_VOLUME_06_{yr:02d}_acft'] = (df[f'ET_Reference_06_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETO_VOLUME_07_{yr:02d}_acft'] = (df[f'ET_Reference_07_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETO_VOLUME_08_{yr:02d}_acft'] = (df[f'ET_Reference_08_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETO_VOLUME_09_{yr:02d}_acft'] = (df[f'ET_Reference_09_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'ETO_VOLUME_10_{yr:02d}_acft'] = (df[f'ET_Reference_10_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
 
    if yr == 0:
        df[f'PPT_VOLUME_11_99_acft'] = (df[f'PPT_11_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
        df[f'PPT_VOLUME_12_99_acft'] = (df[f'PPT_12_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']    
    else:
        df[f'PPT_VOLUME_11_{yr-1:02d}_acft'] = (df[f'PPT_11_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
        df[f'PPT_VOLUME_12_{yr-1:02d}_acft'] = (df[f'PPT_12_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'PPT_VOLUME_01_{yr:02d}_acft'] = (df[f'PPT_01_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'PPT_VOLUME_02_{yr:02d}_acft'] = (df[f'PPT_02_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'PPT_VOLUME_03_{yr:02d}_acft'] = (df[f'PPT_03_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'PPT_VOLUME_04_{yr:02d}_acft'] = (df[f'PPT_04_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'PPT_VOLUME_05_{yr:02d}_acft'] = (df[f'PPT_05_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'PPT_VOLUME_06_{yr:02d}_acft'] = (df[f'PPT_06_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'PPT_VOLUME_07_{yr:02d}_acft'] = (df[f'PPT_07_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'PPT_VOLUME_08_{yr:02d}_acft'] = (df[f'PPT_08_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'PPT_VOLUME_09_{yr:02d}_acft'] = (df[f'PPT_09_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'PPT_VOLUME_10_{yr:02d}_acft'] = (df[f'PPT_10_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']

    if yr == 0:
        df[f'EFF_VOLUME_11_99_acft'] = (df[f'{eff_ppt_var}_11_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
        df[f'EFF_VOLUME_12_99_acft'] = (df[f'{eff_ppt_var}_12_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']    
    else:
        df[f'EFF_VOLUME_11_{yr-1:02d}_acft'] = (df[f'{eff_ppt_var}_11_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
        df[f'EFF_VOLUME_12_{yr-1:02d}_acft'] = (df[f'{eff_ppt_var}_12_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'EFF_VOLUME_01_{yr:02d}_acft'] = (df[f'{eff_ppt_var}_01_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'EFF_VOLUME_02_{yr:02d}_acft'] = (df[f'{eff_ppt_var}_02_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'EFF_VOLUME_03_{yr:02d}_acft'] = (df[f'{eff_ppt_var}_03_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'EFF_VOLUME_04_{yr:02d}_acft'] = (df[f'{eff_ppt_var}_04_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'EFF_VOLUME_05_{yr:02d}_acft'] = (df[f'{eff_ppt_var}_05_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'EFF_VOLUME_06_{yr:02d}_acft'] = (df[f'{eff_ppt_var}_06_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'EFF_VOLUME_07_{yr:02d}_acft'] = (df[f'{eff_ppt_var}_07_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'EFF_VOLUME_08_{yr:02d}_acft'] = (df[f'{eff_ppt_var}_08_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'EFF_VOLUME_09_{yr:02d}_acft'] = (df[f'{eff_ppt_var}_09_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']
    df[f'EFF_VOLUME_10_{yr:02d}_acft'] = (df[f'{eff_ppt_var}_10_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}']

    # cap the effective ppt from ET Demands to the max (total precip) from the field averaged gridMET ppt
    # if yr == 0:
    #     df.loc[df[f'EFF_VOLUME_11_99_acft'] > df[f'PPT_VOLUME_11_99_acft'], f'EFF_VOLUME_11_99_acft'] = df[f'PPT_VOLUME_11_99_acft']
    #     df.loc[df[f'EFF_VOLUME_12_99_acft'] > df[f'PPT_VOLUME_11_99_acft'], f'EFF_VOLUME_11_99_acft'] = df[f'PPT_VOLUME_11_99_acft']    
    # else:
    #     df.loc[df[f'EFF_VOLUME_11_{yr-1:02d}_acft'] > df[f'PPT_VOLUME_11_{yr-1:02d}_acft'], f'EFF_VOLUME_11_{yr-1:02d}_acft'] = df[f'PPT_VOLUME_11_{yr-1:02d}_acft']
    #     df.loc[df[f'EFF_VOLUME_12_{yr-1:02d}_acft'] > df[f'PPT_VOLUME_11_{yr-1:02d}_acft'], f'EFF_VOLUME_11_{yr-1:02d}_acft'] = df[f'PPT_VOLUME_11_{yr-1:02d}_acft']
    # df.loc[df[f'EFF_VOLUME_01_{yr:02d}_acft'] > df[f'PPT_VOLUME_01_{yr:02d}_acft'], f'EFF_VOLUME_01_{yr:02d}_acft'] = df[f'PPT_VOLUME_01_{yr:02d}_acft']
    # df.loc[df[f'EFF_VOLUME_02_{yr:02d}_acft'] > df[f'PPT_VOLUME_02_{yr:02d}_acft'], f'EFF_VOLUME_02_{yr:02d}_acft'] = df[f'PPT_VOLUME_02_{yr:02d}_acft']
    # df.loc[df[f'EFF_VOLUME_03_{yr:02d}_acft'] > df[f'PPT_VOLUME_03_{yr:02d}_acft'], f'EFF_VOLUME_03_{yr:02d}_acft'] = df[f'PPT_VOLUME_03_{yr:02d}_acft']
    # df.loc[df[f'EFF_VOLUME_04_{yr:02d}_acft'] > df[f'PPT_VOLUME_04_{yr:02d}_acft'], f'EFF_VOLUME_04_{yr:02d}_acft'] = df[f'PPT_VOLUME_04_{yr:02d}_acft']
    # df.loc[df[f'EFF_VOLUME_05_{yr:02d}_acft'] > df[f'PPT_VOLUME_05_{yr:02d}_acft'], f'EFF_VOLUME_05_{yr:02d}_acft'] = df[f'PPT_VOLUME_05_{yr:02d}_acft']
    # df.loc[df[f'EFF_VOLUME_06_{yr:02d}_acft'] > df[f'PPT_VOLUME_06_{yr:02d}_acft'], f'EFF_VOLUME_06_{yr:02d}_acft'] = df[f'PPT_VOLUME_06_{yr:02d}_acft']
    # df.loc[df[f'EFF_VOLUME_07_{yr:02d}_acft'] > df[f'PPT_VOLUME_07_{yr:02d}_acft'], f'EFF_VOLUME_07_{yr:02d}_acft'] = df[f'PPT_VOLUME_07_{yr:02d}_acft']
    # df.loc[df[f'EFF_VOLUME_08_{yr:02d}_acft'] > df[f'PPT_VOLUME_08_{yr:02d}_acft'], f'EFF_VOLUME_08_{yr:02d}_acft'] = df[f'PPT_VOLUME_08_{yr:02d}_acft']
    # df.loc[df[f'EFF_VOLUME_09_{yr:02d}_acft'] > df[f'PPT_VOLUME_09_{yr:02d}_acft'], f'EFF_VOLUME_09_{yr:02d}_acft'] = df[f'PPT_VOLUME_09_{yr:02d}_acft']
    # df.loc[df[f'EFF_VOLUME_10_{yr:02d}_acft'] > df[f'PPT_VOLUME_10_{yr:02d}_acft'], f'EFF_VOLUME_10_{yr:02d}_acft'] = df[f'PPT_VOLUME_10_{yr:02d}_acft']
    
    # calculate the consumptive use by subtracting effective ppt from actual et
    if yr == 0:
        df[f'IRR_CU_VOLUME_11_99_acft'] = df[f'ET_VOLUME_11_99_acft'] - df[f'EFF_VOLUME_11_99_acft']
        df[f'IRR_CU_VOLUME_12_99_acft'] = df[f'ET_VOLUME_12_99_acft'] - df[f'EFF_VOLUME_12_99_acft']        
    else:
        df[f'IRR_CU_VOLUME_11_{yr-1:02d}_acft'] = df[f'ET_VOLUME_11_{yr-1:02d}_acft'] - df[f'EFF_VOLUME_11_{yr-1:02d}_acft']
        df[f'IRR_CU_VOLUME_12_{yr-1:02d}_acft'] = df[f'ET_VOLUME_12_{yr-1:02d}_acft'] - df[f'EFF_VOLUME_12_{yr-1:02d}_acft']
    df[f'IRR_CU_VOLUME_01_{yr:02d}_acft'] = df[f'ET_VOLUME_01_{yr:02d}_acft'] - df[f'EFF_VOLUME_01_{yr:02d}_acft']
    df[f'IRR_CU_VOLUME_02_{yr:02d}_acft'] = df[f'ET_VOLUME_02_{yr:02d}_acft'] - df[f'EFF_VOLUME_02_{yr:02d}_acft']
    df[f'IRR_CU_VOLUME_03_{yr:02d}_acft'] = df[f'ET_VOLUME_03_{yr:02d}_acft'] - df[f'EFF_VOLUME_03_{yr:02d}_acft']
    df[f'IRR_CU_VOLUME_04_{yr:02d}_acft'] = df[f'ET_VOLUME_04_{yr:02d}_acft'] - df[f'EFF_VOLUME_04_{yr:02d}_acft']
    df[f'IRR_CU_VOLUME_05_{yr:02d}_acft'] = df[f'ET_VOLUME_05_{yr:02d}_acft'] - df[f'EFF_VOLUME_05_{yr:02d}_acft']
    df[f'IRR_CU_VOLUME_06_{yr:02d}_acft'] = df[f'ET_VOLUME_06_{yr:02d}_acft'] - df[f'EFF_VOLUME_06_{yr:02d}_acft']
    df[f'IRR_CU_VOLUME_07_{yr:02d}_acft'] = df[f'ET_VOLUME_07_{yr:02d}_acft'] - df[f'EFF_VOLUME_07_{yr:02d}_acft']
    df[f'IRR_CU_VOLUME_08_{yr:02d}_acft'] = df[f'ET_VOLUME_08_{yr:02d}_acft'] - df[f'EFF_VOLUME_08_{yr:02d}_acft']
    df[f'IRR_CU_VOLUME_09_{yr:02d}_acft'] = df[f'ET_VOLUME_09_{yr:02d}_acft'] - df[f'EFF_VOLUME_09_{yr:02d}_acft']
    df[f'IRR_CU_VOLUME_10_{yr:02d}_acft'] = df[f'ET_VOLUME_10_{yr:02d}_acft'] - df[f'EFF_VOLUME_10_{yr:02d}_acft']

    if yr == 0:
        df[f'NIWR_VOLUME_11_99_acft'] = ((df[f'NIWR_11_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
        df[f'NIWR_VOLUME_12_99_acft'] = ((df[f'NIWR_12_99_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])   
    else:
        df[f'NIWR_VOLUME_11_{yr-1:02d}_acft'] = ((df[f'NIWR_11_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
        df[f'NIWR_VOLUME_12_{yr-1:02d}_acft'] = ((df[f'NIWR_12_{yr-1:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
    df[f'NIWR_VOLUME_01_{yr:02d}_acft'] = ((df[f'NIWR_01_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
    df[f'NIWR_VOLUME_02_{yr:02d}_acft'] = ((df[f'NIWR_02_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
    df[f'NIWR_VOLUME_03_{yr:02d}_acft'] = ((df[f'NIWR_03_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
    df[f'NIWR_VOLUME_04_{yr:02d}_acft'] = ((df[f'NIWR_04_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
    df[f'NIWR_VOLUME_05_{yr:02d}_acft'] = ((df[f'NIWR_05_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
    df[f'NIWR_VOLUME_06_{yr:02d}_acft'] = ((df[f'NIWR_06_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
    df[f'NIWR_VOLUME_07_{yr:02d}_acft'] = ((df[f'NIWR_07_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
    df[f'NIWR_VOLUME_08_{yr:02d}_acft'] = ((df[f'NIWR_08_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
    df[f'NIWR_VOLUME_09_{yr:02d}_acft'] = ((df[f'NIWR_09_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])
    df[f'NIWR_VOLUME_10_{yr:02d}_acft'] = ((df[f'NIWR_10_{yr:02d}_in'] / 12) * df[f'ACRES_FTR_GEOM_{yr:02d}'])   
    
    
# regular expression to find columns containing the list of substrings below 
reg1 = '|'.join([f'GEOM_{yr_abr_list[0]:02d}','HUC','OWRD','Region','ITYPE','IRR_EFF','srctype','GRIDMET',f'IRRIGATED_{yr_abr_list[0]:02d}',f'WETLAND_{yr_abr_list[0]:02d}',f'{yr_abr_list[0]:02d}_MODE',f'ETD_{yr_abr_list[0]:02d}',
                 f'ET_Fraction_11_{yr_abr_list[0]-1:02d}',f'ET_Fraction_12_{yr_abr_list[0]-1:02d}',f'ET_Fraction_01_{yr_abr_list[0]:02d}',f'ET_Fraction_02_{yr_abr_list[0]:02d}',
                 f'ET_Fraction_03_{yr_abr_list[0]:02d}',f'ET_Fraction_04_{yr_abr_list[0]:02d}',f'ET_Fraction_05_{yr_abr_list[0]:02d}',f'ET_Fraction_06_{yr_abr_list[0]:02d}',
                 f'ET_Fraction_07_{yr_abr_list[0]:02d}',f'ET_Fraction_08_{yr_abr_list[0]:02d}',f'ET_Fraction_09_{yr_abr_list[0]:02d}',f'ET_Fraction_10_{yr_abr_list[0]:02d}',
                 f'11_{yr_abr_list[0]-1:02d}_in',f'12_{yr_abr_list[0]-1:02d}_in',f'01_{yr_abr_list[0]:02d}_in',f'02_{yr_abr_list[0]:02d}_in',f'03_{yr_abr_list[0]:02d}_in',
                 f'04_{yr_abr_list[0]:02d}_in',f'05_{yr_abr_list[0]:02d}_in',f'06_{yr_abr_list[0]:02d}_in',f'07_{yr_abr_list[0]:02d}_in',f'08_{yr_abr_list[0]:02d}_in',
                 f'09_{yr_abr_list[0]:02d}_in',f'10_{yr_abr_list[0]:02d}_in',f'11_{yr_abr_list[0]-1:02d}_acft',f'12_{yr_abr_list[0]-1:02d}_acft',f'01_{yr_abr_list[0]:02d}_acft',
                 f'02_{yr_abr_list[0]:02d}_acft',f'03_{yr_abr_list[0]:02d}_acft',f'04_{yr_abr_list[0]:02d}_acft',f'05_{yr_abr_list[0]:02d}_acft',
                 f'06_{yr_abr_list[0]:02d}_acft',f'07_{yr_abr_list[0]:02d}_acft',f'08_{yr_abr_list[0]:02d}_acft',f'09_{yr_abr_list[0]:02d}_acft',f'10_{yr_abr_list[0]:02d}_acft',f'{yr_list[0]}'])
reg2 = '|'.join([f'GEOM_{yr_abr_list[1]:02d}','HUC','OWRD','Region','ITYPE','IRR_EFF','srctype','GRIDMET',f'IRRIGATED_{yr_abr_list[1]:02d}',f'WETLAND_{yr_abr_list[1]:02d}',f'{yr_abr_list[1]:02d}_MODE',f'ETD_{yr_abr_list[1]:02d}',
                 f'ET_Fraction_11_{yr_abr_list[1]-1:02d}',f'ET_Fraction_12_{yr_abr_list[1]-1:02d}',f'ET_Fraction_01_{yr_abr_list[1]:02d}',f'ET_Fraction_02_{yr_abr_list[1]:02d}',
                 f'ET_Fraction_03_{yr_abr_list[1]:02d}',f'ET_Fraction_04_{yr_abr_list[1]:02d}',f'ET_Fraction_05_{yr_abr_list[1]:02d}',f'ET_Fraction_06_{yr_abr_list[1]:02d}',
                 f'ET_Fraction_07_{yr_abr_list[1]:02d}',f'ET_Fraction_08_{yr_abr_list[1]:02d}',f'ET_Fraction_09_{yr_abr_list[1]:02d}',f'ET_Fraction_10_{yr_abr_list[1]:02d}',
                 f'11_{yr_abr_list[1]-1:02d}_in',f'12_{yr_abr_list[1]-1:02d}_in',f'01_{yr_abr_list[1]:02d}_in',f'02_{yr_abr_list[1]:02d}_in',f'03_{yr_abr_list[1]:02d}_in',f'04_{yr_abr_list[1]:02d}_in',f'05_{yr_abr_list[1]:02d}_in',
                 f'06_{yr_abr_list[1]:02d}_in',f'07_{yr_abr_list[1]:02d}_in',f'08_{yr_abr_list[1]:02d}_in',f'09_{yr_abr_list[1]:02d}_in',f'10_{yr_abr_list[1]:02d}_in',f'11_{yr_abr_list[1]-1:02d}_acft',f'12_{yr_abr_list[1]-1:02d}_acft',
                 f'01_{yr_abr_list[1]:02d}_acft',f'02_{yr_abr_list[1]:02d}_acft',f'03_{yr_abr_list[1]:02d}_acft',f'04_{yr_abr_list[1]:02d}_acft',f'05_{yr_abr_list[1]:02d}_acft',
                 f'06_{yr_abr_list[1]:02d}_acft',f'07_{yr_abr_list[1]:02d}_acft',f'08_{yr_abr_list[1]:02d}_acft',f'09_{yr_abr_list[1]:02d}_acft',f'10_{yr_abr_list[1]:02d}_acft',f'{yr_list[1]}'])
if 0 in yr_abr_list:
    reg3 = '|'.join([f'GEOM_{yr_abr_list[2]:02d}','HUC','OWRD','Region','ITYPE','IRR_EFF','srctype','GRIDMET',f'IRRIGATED_{yr_abr_list[2]:02d}',f'WETLAND_{yr_abr_list[2]:02d}',f'{yr_abr_list[2]:02d}_MODE',f'ETD_{yr_abr_list[2]:02d}',
                     'ET_Fraction_11_99','ET_Fraction_12_99',f'ET_Fraction_01_{yr_abr_list[2]:02d}',f'ET_Fraction_02_{yr_abr_list[2]:02d}',
                     f'ET_Fraction_03_{yr_abr_list[2]:02d}',f'ET_Fraction_04_{yr_abr_list[2]:02d}',f'ET_Fraction_05_{yr_abr_list[2]:02d}',f'ET_Fraction_06_{yr_abr_list[2]:02d}',
                     f'ET_Fraction_07_{yr_abr_list[2]:02d}',f'ET_Fraction_08_{yr_abr_list[2]:02d}',f'ET_Fraction_09_{yr_abr_list[2]:02d}',f'ET_Fraction_10_{yr_abr_list[2]:02d}',
                     '11_99_in','12_99_in',f'01_{yr_abr_list[2]:02d}_in',f'02_{yr_abr_list[2]:02d}_in',f'03_{yr_abr_list[2]:02d}_in',f'04_{yr_abr_list[2]:02d}_in',f'05_{yr_abr_list[2]:02d}_in',
                     f'06_{yr_abr_list[2]:02d}_in',f'07_{yr_abr_list[2]:02d}_in',f'08_{yr_abr_list[2]:02d}_in',f'09_{yr_abr_list[2]:02d}_in',f'10_{yr_abr_list[2]:02d}_in','11_99_acft','12_99_acft',
                     f'01_{yr_abr_list[2]:02d}_acft',f'02_{yr_abr_list[2]:02d}_acft',f'03_{yr_abr_list[2]:02d}_acft',f'04_{yr_abr_list[2]:02d}_acft',f'05_{yr_abr_list[2]:02d}_acft',
                     f'06_{yr_abr_list[2]:02d}_acft',f'07_{yr_abr_list[2]:02d}_acft',f'08_{yr_abr_list[2]:02d}_acft',f'09_{yr_abr_list[2]:02d}_acft',f'10_{yr_abr_list[2]:02d}_acft',f'{yr_list[2]}'])
else:
    reg3 = '|'.join([f'GEOM_{yr_abr_list[2]:02d}','HUC','OWRD','Region','ITYPE','IRR_EFF','srctype','GRIDMET',f'IRRIGATED_{yr_abr_list[2]:02d}',f'WETLAND_{yr_abr_list[2]:02d}',f'{yr_abr_list[2]:02d}_MODE',f'ETD_{yr_abr_list[2]:02d}',
                     f'ET_Fraction_11_{yr_abr_list[2]-1:02d}',f'ET_Fraction_12_{yr_abr_list[2]-1:02d}',f'ET_Fraction_01_{yr_abr_list[2]:02d}',f'ET_Fraction_02_{yr_abr_list[2]:02d}',
                     f'ET_Fraction_03_{yr_abr_list[2]:02d}',f'ET_Fraction_04_{yr_abr_list[2]:02d}',f'ET_Fraction_05_{yr_abr_list[2]:02d}',f'ET_Fraction_06_{yr_abr_list[2]:02d}',
                     f'ET_Fraction_07_{yr_abr_list[2]:02d}',f'ET_Fraction_08_{yr_abr_list[2]:02d}',f'ET_Fraction_09_{yr_abr_list[2]:02d}',f'ET_Fraction_10_{yr_abr_list[2]:02d}',
                     f'11_{yr_abr_list[2]-1:02d}_in',f'12_{yr_abr_list[2]-1:02d}_in',f'01_{yr_abr_list[2]:02d}_in',f'02_{yr_abr_list[2]:02d}_in',f'03_{yr_abr_list[2]:02d}_in',f'04_{yr_abr_list[2]:02d}_in',f'05_{yr_abr_list[2]:02d}_in',
                     f'06_{yr_abr_list[2]:02d}_in',f'07_{yr_abr_list[2]:02d}_in',f'08_{yr_abr_list[2]:02d}_in',f'09_{yr_abr_list[2]:02d}_in',f'10_{yr_abr_list[2]:02d}_in',f'11_{yr_abr_list[2]-1:02d}_acft',f'12_{yr_abr_list[2]-1:02d}_acft',
                     f'01_{yr_abr_list[2]:02d}_acft',f'02_{yr_abr_list[2]:02d}_acft',f'03_{yr_abr_list[2]:02d}_acft',f'04_{yr_abr_list[2]:02d}_acft',f'05_{yr_abr_list[2]:02d}_acft',
                     f'06_{yr_abr_list[2]:02d}_acft',f'07_{yr_abr_list[2]:02d}_acft',f'08_{yr_abr_list[2]:02d}_acft',f'09_{yr_abr_list[2]:02d}_acft',f'10_{yr_abr_list[2]:02d}_acft',f'{yr_list[2]}'])
reg4 = '|'.join([f'GEOM_{yr_abr_list[3]:02d}','HUC','OWRD','Region','ITYPE','IRR_EFF','srctype','GRIDMET',f'IRRIGATED_{yr_abr_list[3]:02d}',f'WETLAND_{yr_abr_list[3]:02d}',f'{yr_abr_list[3]:02d}_MODE',f'ETD_{yr_abr_list[3]:02d}',
                 f'ET_Fraction_11_{yr_abr_list[3]-1:02d}',f'ET_Fraction_12_{yr_abr_list[3]-1:02d}',f'ET_Fraction_01_{yr_abr_list[3]:02d}',f'ET_Fraction_02_{yr_abr_list[3]:02d}',
                 f'ET_Fraction_03_{yr_abr_list[3]:02d}',f'ET_Fraction_04_{yr_abr_list[3]:02d}',f'ET_Fraction_05_{yr_abr_list[3]:02d}',f'ET_Fraction_06_{yr_abr_list[3]:02d}',
                 f'ET_Fraction_07_{yr_abr_list[3]:02d}',f'ET_Fraction_08_{yr_abr_list[3]:02d}',f'ET_Fraction_09_{yr_abr_list[3]:02d}',f'ET_Fraction_10_{yr_abr_list[3]:02d}',
                 f'11_{yr_abr_list[3]-1:02d}_in',f'12_{yr_abr_list[3]-1:02d}_in',f'01_{yr_abr_list[3]:02d}_in',f'02_{yr_abr_list[3]:02d}_in',f'03_{yr_abr_list[3]:02d}_in',f'04_{yr_abr_list[3]:02d}_in',f'05_{yr_abr_list[3]:02d}_in',
                 f'06_{yr_abr_list[3]:02d}_in',f'07_{yr_abr_list[3]:02d}_in',f'08_{yr_abr_list[3]:02d}_in',f'09_{yr_abr_list[3]:02d}_in',f'10_{yr_abr_list[3]:02d}_in',f'11_{yr_abr_list[3]-1:02d}_acft',f'12_{yr_abr_list[3]-1:02d}_acft',
                 f'01_{yr_abr_list[3]:02d}_acft',f'02_{yr_abr_list[3]:02d}_acft',f'03_{yr_abr_list[3]:02d}_acft',f'04_{yr_abr_list[3]:02d}_acft',f'05_{yr_abr_list[3]:02d}_acft',
                 f'06_{yr_abr_list[3]:02d}_acft',f'07_{yr_abr_list[3]:02d}_acft',f'08_{yr_abr_list[3]:02d}_acft',f'09_{yr_abr_list[3]:02d}_acft',f'10_{yr_abr_list[3]:02d}_acft',f'{yr_list[3]}'])
reg5 = '|'.join([f'GEOM_{yr_abr_list[4]:02d}','HUC','OWRD','Region','ITYPE','IRR_EFF','srctype','GRIDMET',f'IRRIGATED_{yr_abr_list[4]:02d}',f'WETLAND_{yr_abr_list[4]:02d}',f'{yr_abr_list[4]:02d}_MODE',f'ETD_{yr_abr_list[4]:02d}',
                 f'ET_Fraction_11_{yr_abr_list[4]-1:02d}',f'ET_Fraction_12_{yr_abr_list[4]-1:02d}',f'ET_Fraction_01_{yr_abr_list[4]:02d}',f'ET_Fraction_02_{yr_abr_list[4]:02d}',
                 f'ET_Fraction_03_{yr_abr_list[4]:02d}',f'ET_Fraction_04_{yr_abr_list[4]:02d}',f'ET_Fraction_05_{yr_abr_list[4]:02d}',f'ET_Fraction_06_{yr_abr_list[4]:02d}',
                 f'ET_Fraction_07_{yr_abr_list[4]:02d}',f'ET_Fraction_08_{yr_abr_list[4]:02d}',f'ET_Fraction_09_{yr_abr_list[4]:02d}',f'ET_Fraction_10_{yr_abr_list[4]:02d}',
                 f'11_{yr_abr_list[4]-1:02d}_in',f'12_{yr_abr_list[4]-1:02d}_in',f'01_{yr_abr_list[4]:02d}_in',f'02_{yr_abr_list[4]:02d}_in',f'03_{yr_abr_list[4]:02d}_in',f'04_{yr_abr_list[4]:02d}_in',f'05_{yr_abr_list[4]:02d}_in',
                 f'06_{yr_abr_list[4]:02d}_in',f'07_{yr_abr_list[4]:02d}_in',f'08_{yr_abr_list[4]:02d}_in',f'09_{yr_abr_list[4]:02d}_in',f'10_{yr_abr_list[4]:02d}_in',f'11_{yr_abr_list[4]-1:02d}_acft',f'12_{yr_abr_list[4]-1:02d}_acft',
                 f'01_{yr_abr_list[4]:02d}_acft',f'02_{yr_abr_list[4]:02d}_acft',f'03_{yr_abr_list[4]:02d}_acft',f'04_{yr_abr_list[4]:02d}_acft',f'05_{yr_abr_list[4]:02d}_acft',
                 f'06_{yr_abr_list[4]:02d}_acft',f'07_{yr_abr_list[4]:02d}_acft',f'08_{yr_abr_list[4]:02d}_acft',f'09_{yr_abr_list[4]:02d}_acft',f'10_{yr_abr_list[4]:02d}_acft',f'{yr_list[4]}'])
reg6 = '|'.join([f'GEOM_{yr_abr_list[5]:02d}','HUC','OWRD','Region','ITYPE','IRR_EFF','srctype','GRIDMET',f'IRRIGATED_{yr_abr_list[5]:02d}',f'WETLAND_{yr_abr_list[5]:02d}',f'{yr_abr_list[5]:02d}_MODE',f'ETD_{yr_abr_list[5]:02d}',
                 f'ET_Fraction_11_{yr_abr_list[5]-1:02d}',f'ET_Fraction_12_{yr_abr_list[5]-1:02d}',f'ET_Fraction_01_{yr_abr_list[5]:02d}',f'ET_Fraction_02_{yr_abr_list[5]:02d}',
                 f'ET_Fraction_03_{yr_abr_list[5]:02d}',f'ET_Fraction_04_{yr_abr_list[5]:02d}',f'ET_Fraction_05_{yr_abr_list[5]:02d}',f'ET_Fraction_06_{yr_abr_list[5]:02d}',
                 f'ET_Fraction_07_{yr_abr_list[5]:02d}',f'ET_Fraction_08_{yr_abr_list[5]:02d}',f'ET_Fraction_09_{yr_abr_list[5]:02d}',f'ET_Fraction_10_{yr_abr_list[5]:02d}',
                 f'11_{yr_abr_list[5]-1:02d}_in',f'12_{yr_abr_list[5]-1:02d}_in',f'01_{yr_abr_list[5]:02d}_in',f'02_{yr_abr_list[5]:02d}_in',f'03_{yr_abr_list[5]:02d}_in',f'04_{yr_abr_list[5]:02d}_in',f'05_{yr_abr_list[5]:02d}_in',
                 f'06_{yr_abr_list[5]:02d}_in',f'07_{yr_abr_list[5]:02d}_in',f'08_{yr_abr_list[5]:02d}_in',f'09_{yr_abr_list[5]:02d}_in',f'10_{yr_abr_list[5]:02d}_in',f'11_{yr_abr_list[5]-1:02d}_acft',f'12_{yr_abr_list[5]-1:02d}_acft',
                 f'01_{yr_abr_list[5]:02d}_acft',f'02_{yr_abr_list[5]:02d}_acft',f'03_{yr_abr_list[5]:02d}_acft',f'04_{yr_abr_list[5]:02d}_acft',f'05_{yr_abr_list[5]:02d}_acft',
                 f'06_{yr_abr_list[5]:02d}_acft',f'07_{yr_abr_list[5]:02d}_acft',f'08_{yr_abr_list[5]:02d}_acft',f'09_{yr_abr_list[5]:02d}_acft',f'10_{yr_abr_list[5]:02d}_acft',f'{yr_list[5]}'])
reg7 = '|'.join([f'GEOM_{yr_abr_list[6]:02d}','HUC','OWRD','Region','ITYPE','IRR_EFF','srctype','GRIDMET',f'IRRIGATED_{yr_abr_list[6]:02d}',f'WETLAND_{yr_abr_list[6]:02d}'f'{yr_abr_list[6]:02d}_MODE',f'ETD_{yr_abr_list[6]:02d}',
                 f'ET_Fraction_11_{yr_abr_list[6]-1:02d}',f'ET_Fraction_12_{yr_abr_list[6]-1:02d}',f'ET_Fraction_01_{yr_abr_list[6]:02d}',f'ET_Fraction_02_{yr_abr_list[6]:02d}',
                 f'ET_Fraction_03_{yr_abr_list[6]:02d}',f'ET_Fraction_04_{yr_abr_list[6]:02d}',f'ET_Fraction_05_{yr_abr_list[6]:02d}',f'ET_Fraction_06_{yr_abr_list[6]:02d}',
                 f'ET_Fraction_07_{yr_abr_list[6]:02d}',f'ET_Fraction_08_{yr_abr_list[6]:02d}',f'ET_Fraction_09_{yr_abr_list[6]:02d}',f'ET_Fraction_10_{yr_abr_list[6]:02d}',
                 f'11_{yr_abr_list[6]-1:02d}_in',f'12_{yr_abr_list[6]-1:02d}_in',f'01_{yr_abr_list[6]:02d}_in',f'02_{yr_abr_list[6]:02d}_in',f'03_{yr_abr_list[6]:02d}_in',f'04_{yr_abr_list[6]:02d}_in',f'05_{yr_abr_list[6]:02d}_in',
                 f'06_{yr_abr_list[6]:02d}_in',f'07_{yr_abr_list[6]:02d}_in',f'08_{yr_abr_list[6]:02d}_in',f'09_{yr_abr_list[6]:02d}_in',f'10_{yr_abr_list[6]:02d}_in',f'11_{yr_abr_list[6]-1:02d}_acft',f'12_{yr_abr_list[6]-1:02d}_acft',
                 f'01_{yr_abr_list[6]:02d}_acft',f'02_{yr_abr_list[6]:02d}_acft',f'03_{yr_abr_list[6]:02d}_acft',f'04_{yr_abr_list[6]:02d}_acft',f'05_{yr_abr_list[6]:02d}_acft',
                 f'06_{yr_abr_list[6]:02d}_acft',f'07_{yr_abr_list[6]:02d}_acft',f'08_{yr_abr_list[6]:02d}_acft',f'09_{yr_abr_list[6]:02d}_acft',f'10_{yr_abr_list[6]:02d}_acft',f'{yr_list[6]}'])

# use regex matches to extract columsn for output files
df1o = df.loc[:,df.columns.str.contains(reg1)]
df2o = df.loc[:,df.columns.str.contains(reg2)]
df3o = df.loc[:,df.columns.str.contains(reg3)]
df4o = df.loc[:,df.columns.str.contains(reg4)]
df5o = df.loc[:,df.columns.str.contains(reg5)]
df6o = df.loc[:,df.columns.str.contains(reg6)]

# remove duplicate columns (static attributes)
df1o = df1o.loc[:,~df1o.columns.duplicated()].copy()
df2o = df2o.loc[:,~df2o.columns.duplicated()].copy()
df3o = df3o.loc[:,~df3o.columns.duplicated()].copy()
df4o = df4o.loc[:,~df4o.columns.duplicated()].copy()
df5o = df5o.loc[:,~df5o.columns.duplicated()].copy()
df6o = df6o.loc[:,~df6o.columns.duplicated()].copy()

df1o = df1o.reset_index()
df2o = df2o.reset_index()
df3o = df3o.reset_index()
df4o = df4o.reset_index()
df5o = df5o.reset_index()
df6o = df6o.reset_index()

# export files to CSV's
df1o.to_csv(os.path.join(out_path, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[0]}_gap_filled.csv'), index=False)
df2o.to_csv(os.path.join(out_path, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[1]}_gap_filled.csv'), index=False)
df3o.to_csv(os.path.join(out_path, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[2]}_gap_filled.csv'), index=False)
df4o.to_csv(os.path.join(out_path, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[3]}_gap_filled.csv'), index=False)
df5o.to_csv(os.path.join(out_path, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[4]}_gap_filled.csv'), index=False)
df6o.to_csv(os.path.join(out_path, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[5]}_gap_filled.csv'), index=False)

# additional year of processing done for certain windows
if (yr_list[0] == 1985 and yr_list[-1] == 1991) or (yr_list[0] == 2016 and yr_list[-1] == 2022):
    
    df7o = df.loc[:,df.columns.str.contains(reg7)]
    
    df7o = df7o.loc[:,~df7o.columns.duplicated()].copy()
    
    df7o = df7o.reset_index()

    df7o.to_csv(os.path.join(out_path, f'or_openet_etdemands_monthly_water_year_shift_1mo_{yr_list[6]}_gap_filled.csv'), index=False)

print('exported all annual files')

## 4. Soil moisture carry forward and applied water calculations

In [ ]:

in_path = table_path.replace('ee_exports', 'post_processing\\4_gap_filled')

out_path = table_path.replace('ee_exports', 'post_processing\\5_field_geodatabase')

# list of years based on start/end year parameters
year_list = list(range(start_year, end_year))

# list of months to loop through
mo_list = [11,12,1,2,3,4,5,6,7,8,9,10]

for year in year_lst:

    df = pd.read_csv(os.path.join(in_path, f'or_openet_etdemands_monthly_water_year_shift_1mo_{year}_gap_filled.csv'), index_col='OPENET_ID')
    
    if year == 1985:
        # have to fill/remake a column each year to carry forward the cumulative WS
        df_cf = df[['ACRES_FTR_GEOM_85', 'HUC8']].copy()

        df_cf['WS_C_Carry_Forward'] = np.nan
        
    if year == 1985:
        
        for mo in mo_list:

            if (mo == 11 or mo == 12):

                # WS is equal to Net ET when Net ET is negative, overwrite positive WS values below
                df[f'WS_{mo}_{str(year-1)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft']

                # if Net ET is positive, overwrite WS to be 0
                df.loc[df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft'] > 0, f'WS_{mo}_{str(year-1)[2:]}_acft'] = 0

                if (year == 1985 and mo == 11):

                    # cumulative WS at first time step is equal to WS
                    df[f'WS_C_{mo}_{str(year-1)[2:]}_acft'] = df[f'WS_{mo}_{str(year-1)[2:]}_acft']

                    # overwrite cumulative WS to be 0 when Net ET is positive
                    df.loc[df['IRR_CU_VOLUME_11_84_acft'] > 0, 'WS_C_11_84_acft'] = 0

                    # new columns to account for adjustments
                    df['IRR_CU_VOLUMEadj_11_84_acft'] = df['IRR_CU_VOLUME_11_84_acft']
                    df.loc[df['IRR_CU_VOLUME_11_84_acft'] < 0, 'IRR_CU_VOLUMEadj_11_84_acft'] = 0
                    
                    df['EFF_VOLUMEadj_11_84_acft'] = df['ET_VOLUME_11_84_acft'] - df['IRR_CU_VOLUMEadj_11_84_acft']
                    
                    # applied water at first time step is set to 0 unless we have a positive Net ET below
                    df['AW_11_84_acft'] = 0

                    # overwrite AW when Net ET is positive 
                    df.loc[df['IRR_CU_VOLUME_11_84_acft'] > 0, 'AW_11_84_acft'] = df['IRR_CU_VOLUME_11_84_acft'] / df['IRR_EFF']

                    # overwrite AW when irrigation efficiency is 0
                    df.loc[df['IRR_EFF'] == 0, 'AW_11_84_acft'] = np.nan
                    
                    
                else:

                    # cumulative WS set to 0
                    df[f'WS_C_{mo}_{str(year-1)[2:]}_acft'] = 0

                    # if the cumulative WS from previous month plus the current Net ET is negative, current cumulative WS is the sum of previous cumulative WS and current Net ET
                    df.loc[(df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft']) < 0, f'WS_C_{mo}_{str(year-1)[2:]}_acft'] = df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft']

                    # new columns to account for adjustments
                    df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft'] + df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft']
                    df.loc[(df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft'] + df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft']) < 0, f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] = 0
                    
                    df[f'EFF_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] = df[f'ET_VOLUME_{mo}_{str(year-1)[2:]}_acft'] - df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft']
                    
                    # set AW to be 0
                    df[f'AW_{mo}_{str(year-1)[2:]}_acft'] = 0

                    # if the current Net ET is positive, AW is calculated
                    df.loc[df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] > 0, f'AW_{mo}_{str(year-1)[2:]}_acft'] = df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] / df['IRR_EFF']

                    # overwrite negative applied water values with 0
                    df.loc[df[f'AW_{mo}_{str(year-1)[2:]}_acft'] < 0, f'AW_{mo}_{str(year-1)[2:]}_acft'] = 0
            
                    # overwrite AW when irrigation efficiency is 0
                    df.loc[df['IRR_EFF'] == 0, f'AW_{mo}_{str(year-1)[2:]}_acft'] = np.nan

            else:
                # WS is equal to Net ET when Net ET is negative, overwrite positive WS values below
                df[f'WS_{mo:02d}_{str(year)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft']

                # if Net ET is positive, overwrite WS to be 0
                df.loc[df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] > 0, f'WS_{mo:02d}_{str(year)[2:]}_acft'] = 0

                # cumulative WS at first time step is equal to WS
                df[f'WS_C_{mo:02d}_{str(year)[2:]}_acft'] = 0

                # if the cumulative WS from previous month plus the current Net ET is negative, current cumulative WS is the sum of previous cumulative WS and current Net ET
                if mo == 1:
                    # january needs to grab a value from previous year
                    df.loc[(df[f'WS_C_12_{str(year-1)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft']) < 0, f'WS_C_{mo:02d}_{str(year)[2:]}_acft'] = df[f'WS_C_12_{str(year-1)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft']
                    
                    # new columns to account for adjustments
                    df[f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_12_{str(year-1)[2:]}_acft']
                    df.loc[(df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_12_{str(year-1)[2:]}_acft']) < 0, f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] = 0
                else:   
                    df.loc[(df[f'WS_C_{(mo-1):02d}_{str(year)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft']) < 0, f'WS_C_{mo:02d}_{str(year)[2:]}_acft'] = df[f'WS_C_{(mo-1):02d}_{str(year)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft']

                    # new columns to account for adjustments
                    df[f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_{(mo-1):02d}_{str(year)[2:]}_acft']
                    df.loc[(df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_{(mo-1):02d}_{str(year)[2:]}_acft']) < 0, f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] = 0
                    

                df[f'EFF_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] = df[f'ET_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] - df[f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft']                    
                    
                # set AW to be 0
                df[f'AW_{mo:02d}_{str(year)[2:]}_acft'] = 0

                # if the current Net ET is positive, AW is calculated
                df.loc[df[f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] > 0, f'AW_{mo:02d}_{str(year)[2:]}_acft'] = df[f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] / df['IRR_EFF']

                # overwrite negative applied water values with 0
                df.loc[df[f'AW_{mo:02d}_{str(year)[2:]}_acft'] < 0, f'AW_{mo:02d}_{str(year)[2:]}_acft'] = 0
                
                # overwrite AW when irrigation efficiency is 0
                df.loc[df['IRR_EFF'] == 0, f'AW_{mo:02d}_{str(year)[2:]}_acft'] = np.nan

                if mo == 10:
                    df_cf['WS_C_Carry_Forward'] = df['WS_C_10_85_acft']
                
    else:
        
        for mo in mo_list:

            if mo == 11:

                # WS is equal to Net ET when Net ET is negative, overwrite positive WS values below
                df[f'WS_{mo}_{str(year-1)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft']

                # if Net ET is positive, overwrite WS to be 0
                df.loc[df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft'] > 0, f'WS_{mo}_{str(year-1)[2:]}_acft'] = 0

                # cumulative WS for previous month set to the previous year's carry forward (Oct of the same year)
                df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft'] = df_cf['WS_C_Carry_Forward']

                # set initial cumulative WS and then check do the check below
                df[f'WS_C_{mo}_{str(year-1)[2:]}_acft'] = 0
                
                # if the cumulative WS from previous month plus the current Net ET is negative, current cumulative WS is the sum of previous cumulative WS and current Net ET
                df.loc[(df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft']) < 0, f'WS_C_{mo}_{str(year-1)[2:]}_acft'] = df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft']

                # new columns to account for adjustments
                df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft'] + df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft']
                df.loc[(df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft'] + df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft']) < 0, f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] = 0

                df[f'EFF_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] = df[f'ET_VOLUME_{mo}_{str(year-1)[2:]}_acft'] - df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft']

                
                # set AW to be 0
                df[f'AW_{mo}_{str(year-1)[2:]}_acft'] = 0

                # if the current Net ET is positive, AW is calculated
                df.loc[df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] > 0, f'AW_{mo}_{str(year-1)[2:]}_acft'] = df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] / df['IRR_EFF']

                # overwrite negative applied water values with 0
                df.loc[df[f'AW_{mo}_{str(year-1)[2:]}_acft'] < 0, f'AW_{mo}_{str(year-1)[2:]}_acft'] = 0
                
                # overwrite AW when irrigation efficiency is 0
                df.loc[df['IRR_EFF'] == 0, f'AW_{mo}_{str(year-1)[2:]}_acft'] = np.nan
                
            elif mo == 12:
                
                # WS is equal to Net ET when Net ET is negative, overwrite positive WS values below
                df[f'WS_{mo}_{str(year-1)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft']

                # if Net ET is positive, overwrite WS to be 0
                df.loc[df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft'] > 0, f'WS_{mo}_{str(year-1)[2:]}_acft'] = 0

                # cumulative WS set to 0
                df[f'WS_C_{mo}_{str(year-1)[2:]}_acft'] = 0

                # if the cumulative WS from previous month plus the current Net ET is negative, current cumulative WS is the sum of previous cumulative WS and current Net ET
                df.loc[(df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft']) < 0, f'WS_C_{mo}_{str(year-1)[2:]}_acft'] = df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft']

                # new columns to account for adjustments
                df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft'] + df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft']
                df.loc[(df[f'IRR_CU_VOLUME_{mo}_{str(year-1)[2:]}_acft'] + df[f'WS_C_{mo-1}_{str(year-1)[2:]}_acft']) < 0, f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] = 0

                df[f'EFF_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] = df[f'ET_VOLUME_{mo}_{str(year-1)[2:]}_acft'] - df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft']
                
                
                # set AW to be 0
                df[f'AW_{mo}_{str(year-1)[2:]}_acft'] = 0

                # if the current Net ET is positive, AW is calculated
                df.loc[df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] > 0, f'AW_{mo}_{str(year-1)[2:]}_acft'] = df[f'IRR_CU_VOLUMEadj_{mo}_{str(year-1)[2:]}_acft'] / df['IRR_EFF']

                # overwrite negative applied water values with 0
                df.loc[df[f'AW_{mo}_{str(year-1)[2:]}_acft'] < 0, f'AW_{mo}_{str(year-1)[2:]}_acft'] = 0
                
                # overwrite AW when irrigation efficiency is 0
                df.loc[df['IRR_EFF'] == 0, f'AW_{mo}_{str(year-1)[2:]}_acft'] = np.nan
                
            else:

                # WS is equal to Net ET when Net ET is negative, overwrite positive WS values below
                df[f'WS_{mo:02d}_{str(year)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft']

                # if Net ET is positive, overwrite WS to be 0
                df.loc[df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] > 0, f'WS_{mo:02d}_{str(year)[2:]}_acft'] = 0

                # cumulative WS set to 0
                df[f'WS_C_{mo:02d}_{str(year)[2:]}_acft'] = 0

                # if the cumulative WS from previous month plus the current Net ET is negative, current cumulative WS is the sum of previous cumulative WS and current Net ET
                if mo == 1:
                    # january needs to grab a value from previous year
                    df.loc[(df[f'WS_C_12_{str(year-1)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft']) < 0, f'WS_C_{mo:02d}_{str(year)[2:]}_acft'] = df[f'WS_C_12_{str(year-1)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft']
                
                    # new columns to account for adjustments
                    df[f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_12_{str(year-1)[2:]}_acft']
                    df.loc[(df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_12_{str(year-1)[2:]}_acft']) < 0, f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] = 0
                else:   
                    df.loc[(df[f'WS_C_{(mo-1):02d}_{str(year)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft']) < 0, f'WS_C_{mo:02d}_{str(year)[2:]}_acft'] = df[f'WS_C_{(mo-1):02d}_{str(year)[2:]}_acft'] + df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft']

                    # new columns to account for adjustments
                    df[f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] = df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_{(mo-1):02d}_{str(year)[2:]}_acft']
                    df.loc[(df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_{(mo-1):02d}_{str(year)[2:]}_acft']) < 0, f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] = 0
                    

                df[f'EFF_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft'] = df[f'ET_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] - df[f'IRR_CU_VOLUMEadj_{mo:02d}_{str(year)[2:]}_acft']  
                    
                # set AW to be 0
                df[f'AW_{mo:02d}_{str(year)[2:]}_acft'] = 0

                # if the current Net ET plus the current cumulative WS is positive, AW is calculated
                if mo == 1:
                    df.loc[(df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_{mo:02d}_{str(year)[2:]}_acft']) > 0, f'AW_{mo:02d}_{str(year)[2:]}_acft'] = (df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_12_{str(year-1)[2:]}_acft']) / df['IRR_EFF']
                else:
                    df.loc[(df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_{mo:02d}_{str(year)[2:]}_acft']) > 0, f'AW_{mo:02d}_{str(year)[2:]}_acft'] = (df[f'IRR_CU_VOLUME_{mo:02d}_{str(year)[2:]}_acft'] + df[f'WS_C_{(mo-1):02d}_{str(year)[2:]}_acft']) / df['IRR_EFF']
                
                # overwrite negative applied water values with 0
                df.loc[df[f'AW_{mo:02d}_{str(year)[2:]}_acft'] < 0, f'AW_{mo:02d}_{str(year)[2:]}_acft'] = 0
                
                # overwrite AW when irrigation efficiency is 0
                df.loc[df['IRR_EFF'] == 0, f'AW_{mo:02d}_{str(year)[2:]}_acft'] = np.nan
                
                if mo == 10:
                    
                    df_cf['WS_C_Carry_Forward'] = df[f'WS_C_10_{str(year)[2:]}_acft']
                
    # flip the sign of the water surplus
    df[f'WS_C_11_{str(year-1)[2:]}_acft'] = df[f'WS_C_11_{str(year-1)[2:]}_acft'] * -1
    df[f'WS_C_12_{str(year-1)[2:]}_acft'] = df[f'WS_C_12_{str(year-1)[2:]}_acft'] * -1
    df[f'WS_C_01_{str(year)[2:]}_acft'] = df[f'WS_C_01_{str(year)[2:]}_acft'] * -1
    df[f'WS_C_02_{str(year)[2:]}_acft'] = df[f'WS_C_02_{str(year)[2:]}_acft'] * -1
    df[f'WS_C_03_{str(year)[2:]}_acft'] = df[f'WS_C_03_{str(year)[2:]}_acft'] * -1
    df[f'WS_C_04_{str(year)[2:]}_acft'] = df[f'WS_C_04_{str(year)[2:]}_acft'] * -1
    df[f'WS_C_05_{str(year)[2:]}_acft'] = df[f'WS_C_05_{str(year)[2:]}_acft'] * -1
    df[f'WS_C_06_{str(year)[2:]}_acft'] = df[f'WS_C_06_{str(year)[2:]}_acft'] * -1
    df[f'WS_C_07_{str(year)[2:]}_acft'] = df[f'WS_C_07_{str(year)[2:]}_acft'] * -1
    df[f'WS_C_08_{str(year)[2:]}_acft'] = df[f'WS_C_08_{str(year)[2:]}_acft'] * -1
    df[f'WS_C_09_{str(year)[2:]}_acft'] = df[f'WS_C_09_{str(year)[2:]}_acft'] * -1
    df[f'WS_C_10_{str(year)[2:]}_acft'] = df[f'WS_C_10_{str(year)[2:]}_acft'] * -1
    
    
    if year == 1985:
        df_out = df.drop([f'WS_11_{str(year-1)[2:]}_acft', f'WS_12_{str(year-1)[2:]}_acft', f'WS_01_{str(year)[2:]}_acft',
                          f'WS_02_{str(year)[2:]}_acft', f'WS_03_{str(year)[2:]}_acft', f'WS_04_{str(year)[2:]}_acft',
                          f'WS_05_{str(year)[2:]}_acft', f'WS_06_{str(year)[2:]}_acft', f'WS_07_{str(year)[2:]}_acft',
                          f'WS_08_{str(year)[2:]}_acft', f'WS_09_{str(year)[2:]}_acft', f'WS_10_{str(year)[2:]}_acft'], axis=1)
    else:
        df_out = df.drop([f'WS_11_{str(year-1)[2:]}_acft', f'WS_12_{str(year-1)[2:]}_acft', f'WS_01_{str(year)[2:]}_acft',
                          f'WS_02_{str(year)[2:]}_acft', f'WS_03_{str(year)[2:]}_acft', f'WS_04_{str(year)[2:]}_acft',
                          f'WS_05_{str(year)[2:]}_acft', f'WS_06_{str(year)[2:]}_acft', f'WS_07_{str(year)[2:]}_acft',
                          f'WS_08_{str(year)[2:]}_acft', f'WS_09_{str(year)[2:]}_acft', f'WS_10_{str(year)[2:]}_acft',
                          f'WS_C_10_{str(year-1)[2:]}_acft'], axis=1)
    
    # fill any null values with 0's
    df_out.loc[:, :] = df_out.fillna(0)
    
    df_out = df_out.reset_index()

    df_out.to_csv(os.path.join(out_path, f'or_openet_etdemands_monthly_water_year_shift_1mo_{year}_final.csv'), index=False)
    
    print(f'exported file for {year}')